<a href="https://colab.research.google.com/github/MadhuReddy001/Image-Captioning/blob/main/Spam_filter_for_Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 32.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

In [2]:
!pip install imbalanced-learn

In [3]:
!pip install protobuf==3.20.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.25.0 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.8.0 which is incompatible.


In [1]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2025-01-06 17:57:36--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4&dl=0 [following]
--2025-01-06 17:57:36--  https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdb4f76c379965c1bb2b8ad28df.dl.dropboxusercontent.com/zip_download_get/CFGRtVIv2TazGR2s1nnUoOmw3q1J6NTi8Qru6uJgO6Z0YFy50avoMW0vyC_YwiqNDBGQa1A4EjyloosBXiYhQg5oFtWPG1pSep23zdMmL7eJCg# [following]
--2025-01-06 17:57:37--  https://ucdb4f76c379965c1bb2b8ad28df.dl.dropboxusercontent.com/zip_

In [2]:
import numpy as np
import pandas as pd
import gensim.downloader as api
import nltk

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
filter = pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0', sep=',', encoding='latin-1', header=1)

In [4]:
filter.drop(index=[1306122, 1306123], inplace=True)

In [5]:
filter.drop(columns=['BN'], inplace=True)

In [6]:
word2vec_model = api.load("word2vec-google-news-300")
embedding_index = {}
for word in word2vec_model.key_to_index:
    embedding_index[word] = word2vec_model[word]

In [7]:
df = pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0', sep=',', encoding='latin-1', header=1)

In [8]:
X = df['question_text']
Y = df['target']

In [9]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [10]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train.dropna().astype(int)),
    y=y_train.dropna().astype(int)
)

In [11]:
class_weight_dict = dict(enumerate(class_weights))

In [12]:
sent_lens = [len(word_tokenize(str(x))) for x in x_train]

In [13]:
max_len = 44

In [14]:
tk = Tokenizer(char_level=False, split=' ')

In [15]:
x_train = x_train.astype(str)
x_test = x_test.astype(str)

x_train = x_train.apply(lambda x: x.lower() if isinstance(x, str) else '')
x_test = x_test.apply(lambda x: x.lower() if isinstance(x, str) else '')

In [16]:
tk.fit_on_texts(x_train)

In [17]:
seq_train = tk.texts_to_sequences(x_train)
seq_test = tk.texts_to_sequences(x_test)

In [18]:
vocab_size = len(tk.word_index)

In [19]:
seq_train_matrix = sequence.pad_sequences(seq_train, maxlen=max_len)
seq_test_matrix = sequence.pad_sequences(seq_test, maxlen=max_len)

In [20]:
word2vec_model = api.load("word2vec-google-news-300")

In [21]:
embedding_dim = 300
vocab_size = len(tk.word_index) + 1

In [22]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tk.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]

In [23]:
inputs = Input(name='text_input', shape=[max_len])

In [24]:
embed = Embedding(len(embedding_matrix), 300,
                  input_length=max_len, mask_zero=True,
                  weights=[embedding_matrix], trainable=False)(inputs)

In [25]:
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)

In [26]:
lstm_layer2 = Dense(256, activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)

In [27]:
lstm_layer3 = Dense(128, activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)

In [28]:
dense1 = Dense(25, activation='relu')(drop3)
drop4 = Dropout(0.2)(dense1)

In [29]:
final_layer = Dense(1, activation='sigmoid')(drop4)

In [30]:
model = Model(inputs=inputs, outputs=final_layer)

In [31]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.keras' # Changed .h5 to .keras
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True)

In [32]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [33]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [34]:
max_len = 44
EMBEDDING_DIM = 100
vocab_size = len(tk.word_index)

In [35]:
inputs = Input(name='text_input', shape=[max_len])

In [36]:
embed = Embedding(vocab_size + 1, EMBEDDING_DIM,
                  input_length=max_len, mask_zero=True,
                  trainable=False)(inputs)

In [37]:
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)

In [38]:
lstm_layer2 = Dense(256, activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)

In [39]:
lstm_layer3 = Dense(128, activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)

In [40]:
dense1 = Dense(25, activation='relu')(drop3)
drop4 = Dropout(0.2)(dense1)

In [41]:
final_layer = Dense(1, activation='sigmoid')(drop4)

In [42]:
model = Model(inputs=inputs, outputs=final_layer)

In [43]:
non_nan_indices = ~y_train.isna()
seq_train_matrix = seq_train_matrix[non_nan_indices]
y_train = y_train[non_nan_indices]

In [44]:
ros = RandomOverSampler(random_state=42)
seq_train_matrix_resampled, y_train_resampled = ros.fit_resample(
    seq_train_matrix, y_train
)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [45]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    seq_train_matrix_resampled,
    y_train_resampled,
    epochs=15,
    batch_size=1000,
    validation_data=(seq_test_matrix, y_test),
    callbacks=[earlystop, checkpoint]
)

Epoch 1/15


In [ ]:
best_model = load_model('/content/weights-04-0.2007.keras')

In [ ]:
best_model_weights = load_model('/content/weights-04-0.2007.keras').get_weights()

inputs = Input(name='text_input', shape=[max_len])
embed = Embedding(vocab_size + 1, 300, input_length=max_len, mask_zero=True,
                  weights=[embedding_matrix], trainable=False)(inputs)
lstm_layer1 = LSTM(512, use_cudnn=False)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256, activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128, activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25, activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1, activation='sigmoid')(drop4)

recreated_model = Model(inputs=inputs, outputs=final_layer)

recreated_model.set_weights(best_model_weights)

p = recreated_model.predict(seq_test_matrix)
roc_auc_score(y_test, p)